In [22]:
import requests
import pandas as pd
from datetime import datetime

In [23]:
api_key = ""

def get_match_ids(puuid, continent):
    api_url = "https://"+continent+".api.riotgames.com/lol/match/v5/matches/by-puuid/"+puuid+"/ids?start=\0&count=98&"+"&api_key="+api_key
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids


def get_match_data(match_id, continent):
    api_url = "https://"+continent+".api.riotgames.com/lol/match/v5/matches/"+match_id+"?api_key="+api_key
    resp = requests.get(api_url)
    match_data = resp.json()
    return match_data


def find_player_data(match_data, player_puuid):
    participants = match_data["metadata"]["participants"]
    player_index = participants.index(player_puuid)
    player_data = match_data["info"]["participants"][player_index]
    date = datetime.fromtimestamp(match_data["info"]["gameStartTimestamp"] / 1000)
    return [player_data, date]

In [24]:
def gather_all_data(match_ids, continent, player_puuid):

    data = {"date": [],"champion": [],"kills": [],"deaths": [],"assists": [],"win": []}

    for match_id in match_ids:
        match_data = get_match_data(match_id, continent, api_key)
        player_data = find_player_data(match_data, player_puuid)

        date = player_data[1]
        champion = player_data[0]["championName"]
        k = player_data[0]["kills"]
        d = player_data[0]["deaths"]
        a = player_data[0]["assists"]
        win = player_data[0]["win"]

        data["date"].append(date)
        data["champion"].append(champion)
        data["kills"].append(k)
        data["deaths"].append(d)
        data["assists"].append(a)
        data["win"].append(win)

    df = pd.DataFrame(data)
    df["win"] = df["win"].astype(int)
    df.to_csv("data.csv", index=False)

    return df

In [25]:
gameName = "PNG dyNquedo"
tagLine = "pain"
continent = "americas"

player_puuid_url = "https://"+continent+".api.riotgames.com/riot/account/v1/accounts/by-riot-id/"+gameName+"/"+tagLine+"?api_key="+api_key
resp_puuid = requests.get(player_puuid_url)
player_info = resp_puuid.json()
player_puuid = player_info["puuid"]
# player_info

# player_matches_url = "https://"+continent+".api.riotgames.com/lol/match/v5/matches/by-puuid/"+player_puuid+"/ids?start=0&count=5"+"&type=ranked&api_key="+api_key
# resp_matches = requests.get(player_matches_url)
# matches_ids = resp_matches.json()
# matches_ids

In [26]:
match_ids = get_match_ids(player_puuid, continent, api_key)
gather_all_data(match_ids, continent, player_puuid)

,date,champion,kills,deaths,assists,win
0,2024-02-11 16:26:36.425,Yasuo,12,4,2,1
1,2024-02-10 17:50:54.440,Yone,1,0,0,1
2,2024-02-04 17:48:54.042,Yone,1,2,0,0
3,2024-02-04 17:01:11.065,Akali,14,1,6,1
4,2024-02-04 15:58:00.267,Yasuo,12,2,3,1
...,...,...,...,...,...,...
93,2023-11-14 05:49:59.593,Illaoi,6,5,10,1
94,2023-11-14 05:17:58.316,Quinn,11,6,13,0
95,2023-11-14 04:43:09.162,KogMaw,15,7,18,1
96,2023-11-14 04:10:20.497,Jhin,7,9,12,1
